# Behaviour Cloning Project

## Import Needed Modules

In [1]:
import csv
import cv2
import os

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import Sequential

## Generate Dataset from Raw Data

In [2]:
def datasetGen():
    with open('data/driving_log.csv') as log_file:
        log_reader = csv.DictReader(log_file)
        X = []
        y = []
        steering_offset = 0.4

        for row in log_reader:
            centerImage = mpimg.imread(row['center'].strip().replace('IMG', 'data/IMG'))
            flippedCenterImage = np.fliplr(centerImage)
            centerSteering = float(row['steering'])

            leftImage = mpimg.imread(row['left'].strip().replace('IMG', 'data/IMG'))
            flippedLeftImage = np.fliplr(leftImage)
            leftSteering = centerSteering + steering_offset

            rightImage = mpimg.imread(row['right'].strip().replace('IMG', 'data/IMG'))
            flippedRightImage = np.fliplr(rightImage)
            rightSteering = centerSteering - steering_offset
            
            X.append(centerImage)
            X.append(flippedCenterImage)
            X.append(leftImage)
            X.append(flippedLeftImage)
            X.append(rightImage)
            X.append(flippedRightImage)
            
            y.append(centerSteering)
            y.append(-centerSteering)
            y.append(leftSteering)
            y.append(-leftSteering)
            y.append(rightSteering)
            y.append(-rightSteering)

#            X = np.append(X, 
#                          np.array([centerImage, 
#                                    flippedCenterImage, 
#                                    leftImage, 
#                                    flippedLeftImage, 
#                                    rightImage, 
#                                    flippedRightImage]), 
#                         axis=0)
                        
#            y = np.append(y, 
#                          np.array([centerSteering, 
#                                    -centerSteering, 
#                                    leftSteering, 
#                                    -leftSteering, 
#                                    rightSteering, 
#                                    -rightSteering]), 
#                          axis=0)
    X = np.array(X)
    y = np.array(y)    
    return X, y

In [3]:
X, y = datasetGen()

In [4]:
X_train, X_valid, y_train, y_valid =  train_test_split(X, y, test_size=0.15, random_state=42)

## Network Architecture 

In [ ]:
def model():
    model = Sequential()

    model.add(layers.BatchNormalization(input_shape=(160, 320, 3), 
                                        axis=-1,
                                        momentum=0.99, 
                                        epsilon=0.001, 
                                        center=True, 
                                        scale=True, 
                                        beta_initializer='zeros', 
                                        gamma_initializer='ones', 
                                        moving_mean_initializer='zeros', 
                                        moving_variance_initializer='ones', 
                                        beta_regularizer=None, 
                                        gamma_regularizer=None, 
                                        beta_constraint=None, 
                                        gamma_constraint=None))

    model.add(layers.Conv2D(8, 
                            kernel_size=(5, 5), 
                            strides=(1, 1), 
                            activation='relu', 
                            padding='same'))

    model.add(layers.Conv2D(16, 
                            kernel_size=(5, 5), 
                            strides=(2, 2), 
                            activation='relu', 
                            padding='valid'))

    model.add(layers.MaxPooling2D(pool_size=(2, 2), 
                                      strides=(1, 1), 
                                      padding='valid'))

    model.add(layers.Conv2D(32, 
                            kernel_size=(5, 5), 
                            strides=(2, 2), 
                            activation='relu', 
                            padding='valid'))

    model.add(layers.Conv2D(32, 
                            kernel_size=(3, 3), 
                            strides=(2, 2), 
                            activation='relu', 
                            padding='valid'))

    model.add(layers.MaxPooling2D(pool_size=(4, 4), 
                                      strides=(2, 2), 
                                      padding='valid'))

    model.add(layers.Conv2D(64, 
                            kernel_size=(3, 3), 
                            strides=(1, 1), 
                            activation='relu', 
                            padding='valid'))

    model.add(layers.Conv2D(64, 
                            kernel_size=(3, 3), 
                            strides=(1, 1), 
                            activation='relu', 
                            padding='valid'))

    model.add(layers.Dropout(0.4))

    model.add(layers.Flatten())

    model.add(layers.Dense(1024, activation='linear'))

    model.add(layers.Dense(256, activation='linear'))

    model.add(layers.Dense(64, activation='linear'))

    model.add(layers.Dense(16, activation='linear'))

    model.add(layers.Dense(1, activation='linear'))

    model.compile(loss='mse', optimizer='adam')

    return model

In [ ]:
model = model()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_v1 (Batc (None, 160, 320, 3)       12        
_________________________________________________________________
conv2d (Conv2D)              (None, 160, 320, 8)       608       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 78, 158, 16)       3216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 77, 157, 16)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None,

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=256,
          validation_data=(X_valid, y_valid))

model.save('model.h5')

Train on 40983 samples, validate on 7233 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
40983/40983==============================] - 36s 885us/sample - loss: 1.1017 - val_loss: 0.0223
Epoch 2/5
40983/40983==============================] - 34s 829us/sample - loss: 0.0231 - val_loss: 0.0144
Epoch 3/5
40983/40983==============================] - 34s 829us/sample - loss: 0.0178 - val_loss: 0.0126
Epoch 4/5
40983/40983==============================] - 34s 829us/sample - loss: 0.0156 - val_loss: 0.0115
Epoch 5/5
 8448/40983=====>........................] - ETA: 25s - loss: 0.0154